## MiFID II classification of trades using the RTS 2 Annex 3 taxonomy.

Governments would prefer to avoid another financial crisis like the one in 2008 and beleive that making the big players operate in a more open and transparent way will help avoid another crash.

Markets in Financial Instruments Directive II (MiFID II) is an EU law which has market transparency as its key objective.  The predecessor law, MiFID I, only looked at a part of what banking firms do.  MiFID II aims to cover most mainstream activity.

Governments rely on regulators to make sure that their laws are being followed.  For MiFID II the primary regulator is ESMA.  ESMA have produced a number of Regulatory Technical Standard (RTS) documents which aim to explain what banking firms must do to comply with the MiFID II law.

One of the RTS documents, RTS 2, explains how different kinds of trading activity can be identified.  Having a clear way to say what has been traded is an important part of making the markets more transparent.

Some kinds of trading activity is already pretty transparent, for example buying and selling shares in a public company.  Trades of this kind are mostly done using a public exchange, such as the New York or London stock exchanges.  The 'price' for a given stock is simply the amount of money paid in the most recent trade and this price is made public by the exchange so everyone can see what the latest price is.  It is pretty easy to identify what has been traded because each stock has a identifier, e.g. 'AAPL' identifies Apple Inc. shares.

Not all trades happen on public exchanges.  Many trades happen directly between two parties and these are known as over the counter (OTC) trades.  Each OTC trade can be fine tuned, for example setting payment dates and interest rates.  The fine tuning of OTC deals makes it hard to identify what has been traded, but this is where RTS 2 comes in. 

The easyest way to understand what RTS 2 is all about is to use it to classify some trades, and you can do just that below.


In [191]:
#import sys
#sys.path.append('/home/bbadger/bag/data/dev/mifid2-rts/mifid2-rts/rts')
import rts2_annex3
import datetime

In [192]:
class SampleTrade(object):
    pass

sample_trade = SampleTrade()
sample_trade.asset_class_name = 'Foreign Exchange Derivatives'
sample_trade.sub_asset_class_name= 'Deliverable FX options (DO)'
sample_trade.underlying_currency_pair = ('GBP', 'USD')
sample_trade.from_date = datetime.date(2017, 8, 13)
sample_trade.to_date = datetime.date(2017, 10, 12)

In [193]:
sample_classification = rts2_annex3.class_root.classification_for(sample_trade)

In [194]:
sample_classification.classification_dict()

{'Asset class': 'Foreign Exchange Derivatives',
 'RTS2 version': 'Brussels, 14.7.2016 C(2016) 4301 final ANNEXES 1 to 4',
 'Segmentation criterion 1': "('GBP', 'USD')",
 'Segmentation criterion 1 description': 'underlying currency pair defined as combination of the two currencies underlying the derivative contract',
 'Segmentation criterion 2': 'Maturity bucket 2: 1 week to 3 months',
 'Segmentation criterion 2 description': 'time to maturity bucket of the swap defined as follows:',
 'Sub-asset class': 'Deliverable FX options (DO)'}

In [195]:
import ipywidgets as widgets

In [196]:
asset_classes = rts2_annex3.class_root.asset_classes
asset_class_names = [an_asset_class.name for an_asset_class in asset_classes]
asset_class_dict = dict([
        (an_asset_class.name, an_asset_class) 
        for an_asset_class 
        in asset_classes])

In [197]:
asset_class_widget = widgets.Dropdown(
    options=asset_class_dict,
    # value=asset_class_dict[asset_class_names[0]],
    description='Asset Classes:',
    disabled=False,
)
asset_class_widget

In [198]:
#selected_asset_class = rts2_annex3.class_root.asset_class_by_name(asset_class_widget.value)
selected_asset_class = asset_class_widget.value
sub_asset_class_names = [
    sub_asset_class.name 
    for sub_asset_class 
    in selected_asset_class.sub_asset_classes]
sub_asset_class_dict = dict([
    (sub_asset_class.name, sub_asset_class) 
    for sub_asset_class 
    in selected_asset_class.sub_asset_classes])

In [199]:
sub_asset_class_widget = widgets.Dropdown(
    options=sub_asset_class_dict,
    # value=sub_asset_class_names[0],
    description='Sub-asset Classes:',
    disabled=False,
)
sub_asset_class_widget

In [200]:
criteria_vbox = widgets.VBox([])

In [201]:
from IPython.display import display

In [202]:
def criteria_widgets(sub_asset_class):
    # OK, in here I need to look up the criteria for the
    # sub-asset class and build the widgets in rows of HBox es
    return [
        widgets.Label(sub_asset_class.name + " L1"),
        widgets.Label(sub_asset_class.name + " L2")
    ]
    
def asset_class_changed(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_asset_class = change['new']
        sub_asset_class_names = [sub_asset_class.name for sub_asset_class in selected_asset_class.sub_asset_classes]
        sub_asset_class_dict = dict([
            (sub_asset_class.name, sub_asset_class) 
            for sub_asset_class 
            in selected_asset_class.sub_asset_classes])
        sub_asset_class_widget.options = sub_asset_class_dict
        
def sub_asset_class_changed(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_sub_asset_class = change['new']
        criteria_vbox.children = criteria_widgets(selected_sub_asset_class)

asset_class_widget.observe(asset_class_changed)
sub_asset_class_widget.observe(sub_asset_class_changed)
display(asset_class_widget)
display(sub_asset_class_widget)
criteria_vbox.children = criteria_widgets(sub_asset_class_widget.value)
display(criteria_vbox)


In [203]:
# help(widgets.VBox)
# help(widgets.Label)
l_1 = widgets.Label("L1")
vbox_1 = widgets.VBox([widgets.Label("xx"), l_1])
display(vbox_1)
l_2 = widgets.Label("L2")
vbox_1.children = ([widgets.Label("yy"), l_2])
display(vbox_1)

In [204]:
sub_asset_class_widget.value.parent.name

'C10 derivatives'

In [205]:
from ipywidgets import interact
@interact(x=dict(a=1,b=2,c=3))
def xx(x):
    return x

1

In [206]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)